In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
from statsmodels.stats.outliers_influence import variance_inflation_factor

import warnings
warnings.simplefilter("ignore")

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [3]:
data=pd.read_excel("Folds5x2_pp.xlsx")
data.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


In [ ]:
#AT=AMBIENT TEMPERATURE,AP=ATMOSPHERIC PRESSURE,RH=RELATIVE HUMIDITY,V=EXHAUST VACCUM,PE=NET HOURLY ELECTRICAL ENERGY OUTPUT

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [5]:
data.columns

Index(['AT', 'V', 'AP', 'RH', 'PE'], dtype='object')

In [6]:
continous=['AT', 'V', 'AP', 'RH', 'PE']

In [7]:
data["AT"].unique()

array([14.96, 25.18,  5.11, ..., 30.  ,  5.01,  9.71])

In [8]:
data["V"].unique()

array([41.76, 62.96, 39.4 , 57.32, 37.5 , 59.44, 43.96, 44.71, 45.  ,
       43.56, 43.72, 46.93, 73.5 , 58.59, 69.34, 43.79, 41.74, 52.75,
       38.47, 42.42, 40.07, 42.28, 63.9 , 48.6 , 70.72, 39.31, 39.96,
       35.79, 65.18, 63.94, 58.41, 66.85, 74.16, 44.03, 63.73, 47.45,
       39.35, 51.3 , 44.85, 41.54, 42.86, 40.64, 37.87, 43.43, 40.11,
       38.62, 78.92, 42.18, 39.39, 59.43, 64.44, 39.33, 61.08, 58.84,
       52.3 , 65.71, 40.1 , 45.87, 58.79, 65.34, 40.69, 34.03, 74.34,
       68.3 , 41.01, 74.67, 61.02, 39.85, 69.75, 67.25, 76.86, 69.45,
       43.02, 45.08, 73.68, 39.3 , 58.96, 68.94, 51.43, 64.05, 60.95,
       41.66, 52.72, 67.32, 40.72, 66.48, 63.77, 59.21, 43.69, 41.38,
       71.94, 62.66, 49.69, 44.2 , 65.59, 40.89, 61.87, 58.33, 39.72,
       43.48, 46.21, 59.39, 41.1 , 66.93, 42.85, 50.88, 54.2 , 68.67,
       73.18, 73.88, 60.84, 45.09, 57.76, 43.77, 63.76, 47.43, 62.91,
       39.04, 44.78, 69.05, 59.8 , 65.06, 68.14, 42.32, 40.77, 62.52,
       48.41, 39.9 ,

In [9]:
data["AP"].unique()

array([1024.07, 1020.04, 1012.16, ..., 1027.12, 1024.16, 1023.67])

In [10]:
data["RH"].unique()

array([73.17, 59.08, 92.14, ..., 94.03, 63.83, 36.48])

In [11]:
data["PE"].unique()

array([463.26, 444.37, 488.56, ..., 484.33, 443.21, 469.62])

In [12]:
data["AT"].value_counts()

AT
25.21    14
13.78    12
24.43    11
23.56    10
11.02    10
         ..
9.89      1
7.34      1
12.92     1
27.72     1
9.71      1
Name: count, Length: 2773, dtype: int64

In [13]:
data["AT"].value_counts()

AT
25.21    14
13.78    12
24.43    11
23.56    10
11.02    10
         ..
9.89      1
7.34      1
12.92     1
27.72     1
9.71      1
Name: count, Length: 2773, dtype: int64

In [14]:
data["AT"].value_counts()

AT
25.21    14
13.78    12
24.43    11
23.56    10
11.02    10
         ..
9.89      1
7.34      1
12.92     1
27.72     1
9.71      1
Name: count, Length: 2773, dtype: int64

In [15]:
data["AT"].value_counts()

AT
25.21    14
13.78    12
24.43    11
23.56    10
11.02    10
         ..
9.89      1
7.34      1
12.92     1
27.72     1
9.71      1
Name: count, Length: 2773, dtype: int64

In [16]:
data["AT"].value_counts()

AT
25.21    14
13.78    12
24.43    11
23.56    10
11.02    10
         ..
9.89      1
7.34      1
12.92     1
27.72     1
9.71      1
Name: count, Length: 2773, dtype: int64

In [17]:
def plot_boxplot(data,ft):
    data.boxplot(column=[ft])
    plt.grid(False)
    plt.show()

In [18]:
def outliers(data,ft):
    Q1=data[ft].quantile(0.25)
    Q3=data[ft].quantile(0.75)
    IQR=Q3-Q1
    lower_bound=Q1-1.5*IQR
    upper_bound=Q3+1.5*IQR

    ls=data.index[ (data[ft] < lower_bound) | (data[ft]> upper_bound) ]

    return ls

In [19]:
index_list=[]
for feature in ['AT', 'V', 'AP', 'RH','PE']:
    index_list.extend(outliers(data,feature))
index_list

[238,
 319,
 368,
 536,
 571,
 631,
 669,
 715,
 753,
 810,
 847,
 983,
 1091,
 1250,
 1290,
 1388,
 1583,
 1660,
 1808,
 2003,
 2060,
 2298,
 2372,
 2395,
 2447,
 2481,
 2579,
 2761,
 2972,
 2977,
 2989,
 3015,
 3021,
 3066,
 3213,
 3412,
 3443,
 3483,
 3492,
 3766,
 3796,
 3955,
 4067,
 4125,
 4202,
 4327,
 4697,
 4793,
 4928,
 5006,
 5067,
 5431,
 5529,
 5690,
 5706,
 5805,
 5840,
 5863,
 5928,
 5956,
 6006,
 6346,
 6581,
 6708,
 6789,
 6829,
 6925,
 7077,
 7186,
 7320,
 7485,
 7623,
 7807,
 7892,
 8098,
 8104,
 8133,
 8270,
 8379,
 8411,
 8462,
 8568,
 9060,
 9068,
 9159,
 9267,
 9342,
 9366,
 789,
 2262,
 3603,
 4367,
 6472,
 7983,
 8061,
 8737,
 8751,
 9015,
 9074,
 9075]

In [20]:
#def remove(data,ls):
#  ls=sorted(set(ls))
#   data=data.drop(ls)
#    return data

In [21]:
#data_cleaned=remove(data,index_list)

In [22]:
data.skew()

AT   -0.136393
V     0.198521
AP    0.265445
RH   -0.431839
PE    0.306509
dtype: float64

In [23]:
X=data.drop("PE",axis=1)
y=data["PE"]

In [24]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

from sklearn.linear_model import LinearRegression
model1=LinearRegression()
model1.fit(X_train,y_train)

from sklearn.model_selection import cross_val_score
print("Intercept:",model1.intercept_)
print("coefficients:",model1.coef_)

train_predictions=model1.predict(X_train)
test_predictions=model1.predict(X_test)

print("Train R2:",model1.score(X_train,y_train))
print("Test R2:",model1.score(X_test,y_test))
print("cross validation score:",cross_val_score(model1,X,y,cv=5).mean())

Intercept: 458.3558797966692
coefficients: [-1.97253609 -0.23689539  0.05852478 -0.15902722]
Train R2: 0.9281184327195898
Test R2: 0.9309231890731563
cross validation score: 0.9285378066739305


In [27]:
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("\nVariance Inflation Factor (VIF):\n", vif_data)


Variance Inflation Factor (VIF):
   feature        VIF
0      AT  39.157705
1       V  74.969127
2      AP  66.618618
3      RH  40.704756


In [28]:
import statsmodels.formula.api as smf
model2=smf.ols("y~X",data=data).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.929
Model:                            OLS   Adj. R-squared:                  0.929
Method:                 Least Squares   F-statistic:                 3.114e+04
Date:                Thu, 18 Jul 2024   Prob (F-statistic):               0.00
Time:                        16:07:09   Log-Likelihood:                -28088.
No. Observations:                9568   AIC:                         5.619e+04
Df Residuals:                    9563   BIC:                         5.622e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    454.6093      9.749     46.634      0.000     435.500     473.718
X[0]          -1.9775      0.015   -129.342      0.000      -2.007      -1.948
X[1]          -0.2339      0.007    -32.122      0.000      -0.248      -0.220
X[2]           0.0621      0.009      6.564      0.000       0.044       0.081
X[3]          -0.1581      0.004    -37.918      0.000      -0.166      -0.150
==============================================================================
Omnibus:                      892.002   Durbin-Watson:                   2.033
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4086.777
Skew:                          -0.352   Prob(JB):                         0.00
Kurtosis:                       6.123   Cond. No.                     2.13e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.13e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=13)

In [30]:
from sklearn.preprocessing import PolynomialFeatures
polynomial_converter=PolynomialFeatures(degree=2)
X_train_poly=pd.DataFrame(polynomial_converter.fit_transform(X_train))

from sklearn.linear_model import LinearRegression
model=LinearRegression()
model.fit(X_train_poly,y_train)

ypred_train=model.predict(X_train_poly)

from sklearn.metrics import mean_squared_error,r2_score
print("Train RMSE:",np.sqrt(mean_squared_error(y_train,ypred_train)))
print("Train R2:",r2_score(y_train,ypred_train))

from sklearn.model_selection import cross_val_score
print("Cross Validation Score:",cross_val_score(model,X_train_poly,y_train,cv=5).mean())


Train RMSE: 4.25654379557425
Train R2: 0.9378161472059803
Cross Validation Score: 0.9374750666020747


In [31]:
X_test_poly=pd.DataFrame(polynomial_converter.transform(X_test))

ypred_test=model.predict(X_test_poly)

print("Test RMSE:",np.sqrt(mean_squared_error(y_test,ypred_test)))
print("Test R2:",r2_score(y_test,ypred_test))

Test RMSE: 4.254326301460243
Test R2: 0.9377179628847947


In [32]:
train_r2=[]
test_r2=[]

for i in range(1,10):
    polynomial_converter=PolynomialFeatures(degree=i)
    X_train_poly=pd.DataFrame(polynomial_converter.fit_transform(X_train))

    model=LinearRegression()
    model.fit(X_train_poly,y_train)

    train_pred=model.predict(X_train_poly)
    train_r2.append(model.score(X_train_poly,y_train))

    X_test_poly=pd.DataFrame(polynomial_converter.transform(X_test))

    test_pred=model.predict(X_test_poly)
    test_r2.append(model.score(X_test_poly,y_test))

In [33]:
train_r2

[0.9287417335236683,
 0.9378161472059803,
 0.940729195363161,
 0.9426193654868534,
 0.9446788460661021,
 0.9460333671392918,
 0.9470865370667045,
 0.9464520590666022,
 0.946797910530958]

In [34]:
test_r2

[0.9284252072118927,
 0.9377179628847947,
 0.9411051207322805,
 0.9424874182023198,
 0.9438469086702983,
 0.945829487368187,
 0.9442847336843304,
 0.9424291890514008,
 0.9452069994580989]

In [35]:
final_poly_converter=PolynomialFeatures(degree=6)
X_train_poly=pd.DataFrame(final_poly_converter.fit_transform(X_train))

final_model=LinearRegression()
final_model.fit(X_train_poly,y_train)

train_pred=final_model.predict(X_train_poly)
print("train R2:",final_model.score(X_train_poly,y_train))
print("Cross validation score:",cross_val_score(model,X_train_poly,y_train,cv=5).mean())

X_test_poly=pd.DataFrame(final_poly_converter.transform(X_test))
test_pred=final_model.predict(X_test_poly)
print("test R2:",final_model.score(X_test_poly,y_test))

train R2: 0.9460333671392918
Cross validation score: 0.9426078450507012
test R2: 0.945829487368187


In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

In [37]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
estimator=Lasso()
param_grid={"alpha":list(range(1,100))}

model_hp=GridSearchCV(estimator,param_grid,cv=5,scoring='r2')

model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 1}

In [38]:
from sklearn.linear_model import Lasso
lasso_best=Lasso(alpha=1)
lasso_best.fit(X_train,y_train)

print("Intercept:",lasso_best.intercept_)
print("coefficients:",lasso_best.coef_)

ypred_train=lasso_best.predict(X_train)
from sklearn.metrics import r2_score
print("train r2:",r2_score(y_train,ypred_train))
from sklearn.model_selection import cross_val_score
print("Cv score:",cross_val_score(lasso_best,X_train,y_train,cv=5).mean())

ypred_test=lasso_best.predict(X_test)
print("test r2:",r2_score(y_test,ypred_test))


Intercept: 469.99020855487066
coefficients: [-1.91692691 -0.25504403  0.045803   -0.14336738]
train r2: 0.9278927920705954
Cv score: 0.9277105859223717
test r2: 0.930545862866579


In [39]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
estimator=Ridge()

param_grid={"alpha":list(range(1,100))}

model_hp=GridSearchCV(estimator,param_grid,cv=5,scoring='r2')

model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 86}

In [40]:
ridge_best=Ridge(alpha=86)
ridge_best.fit(X_train,y_train)

print("intercept:",ridge_best.intercept_)
print("coefficients:",ridge_best.coef_)

ypred_train=ridge_best.predict(X_train)
print("train r2:",r2_score(y_train,ypred_train))
print("cv Score:",cross_val_score(ridge_best,X_train,y_train,cv=5).mean())

ypred_test=ridge_best.predict(X_test)
print("test r2:",r2_score(y_test,ypred_test))

intercept: 452.233241377168
coefficients: [-1.97083479 -0.23736736  0.06445296 -0.15768559]
train r2: 0.9277252321177676
cv Score: 0.9274552056083067
test r2: 0.932528360586564


In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=9)

from sklearn.linear_model import ElasticNet
enr_base=ElasticNet()
enr_base.fit(X_train,y_train)

train_predictions=enr_base.predict(X_train)
test_predictions=enr_base.predict(X_test)

print("train R2:",enr_base.score(X_train,y_train))
print("test R2:",enr_base.score(X_test,y_test))
from sklearn.model_selection import cross_val_score
print("cross Validation Score:",cross_val_score(enr_base,X,y,cv=5).mean())

train R2: 0.9276050603307082
test R2: 0.9301809469006899
cross Validation Score: 0.9280334249499027


In [42]:
from sklearn.model_selection import GridSearchCV

estimator=ElasticNet()

param_grid={"alpha":[0.1,0.2,1,2,3,5,10],"l1_ratio":[0.1,0.5,0.75,0.9,0.95,1]}

model_hp=GridSearchCV(estimator,param_grid,cv=5,scoring='neg_mean_squared_error')
model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 0.1, 'l1_ratio': 1}

In [43]:
enr_best=ElasticNet(alpha=0.1,l1_ratio=1)
enr_best.fit(X_train,y_train)

print("intercept:",enr_best.intercept_)
print("coefficients",enr_best.coef_)

train_predictions=enr_best.predict(X_train)
test_predictions=enr_best.predict(X_test)

print("Train R2:",enr_best.score(X_train,y_train))
print("test R2:",enr_best.score(X_test,y_test))
print("cross validation score:",cross_val_score(enr_best,X,y,cv=5).mean())

intercept: 459.6933971005806
coefficients [-1.96757445 -0.23849951  0.05708851 -0.15756332]
Train R2: 0.928116387007527
test R2: 0.9309066744902176
cross validation score: 0.9285358237152195


In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [45]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [46]:
from keras.models import Sequential
from keras.layers import Dense

In [47]:
def ann_model_regressor():
    
    model=Sequential()

    model.add(Dense(input_dim=4,units=6,activation="relu",kernel_initializer="uniform"))

    model.add(Dense(units=6,activation="relu",kernel_initializer="uniform"))

    model.add(Dense(units=1,activation="relu",kernel_intitializer="uniform"))

    model.compile(optimizer="adam",loss="mean_squared_error")

    return model

In [48]:
def ann_model_regressor():
    model=Sequential()
    model.add(Dense(input_dim=4,units=6,activation="relu",kernel_initializer="uniform"))
    model.add(Dense(units=6,activation="relu",kernel_initializer="uniform"))
    model.add(Dense(units=1,activation="relu",kernel_initializer="uniform"))
    model.compile(optimizer="adam",loss="mean_squared_error")
    return model

In [49]:
ann=ann_model_regressor()

In [50]:
ann.fit(X_train,y_train,batch_size=32,epochs=100)

Epoch 1/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 206509.7344
Epoch 2/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 200616.8906
Epoch 3/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 174677.8125
Epoch 4/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 126006.6641
Epoch 5/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 74858.8203
Epoch 6/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 43024.8398
Epoch 7/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27009.7559
Epoch 8/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19032.5293
Epoch 9/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 14241.5537
Epoch 10/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11120.1201
Epoch 11/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8182.8179
Epoch 12/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5719.7178
Epoch 13/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 4007.4802
Epoch 14/100
240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - los

In [51]:
ypred_train=ann.predict(X_train)
ypred_test=ann.predict(X_test)

240/240 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 922us/step


In [52]:
from sklearn.metrics import mean_squared_error
print("MSE for train data:",mean_squared_error(y_train,ypred_train))
print("MSE for test data:",mean_squared_error(y_test,ypred_test))

MSE for train data: 20.702926104924323
MSE for test data: 19.2048234895522


In [53]:
from sklearn.metrics import r2_score
print("R2 for train data ",r2_score(y_train,ypred_train))
print("R2 for test data: ",r2_score(y_test,ypred_test))

R2 for train data  0.9288372575497132
R2 for test data:  0.9343397531163401
